In [3]:
import pandas as pd 

In [4]:
# Load the training and test data
train_data = pd.read_csv('train (1).csv')
test_data = pd.read_csv('test (2).csv')

Validation Accuracy: 0.5215277777777778


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the data
train_data = pd.read_csv('train (1).csv')
test_data = pd.read_csv('test (2).csv')

# Drop irrelevant columns like 'Id', 'Artist Name', 'Track Name'
train_data = train_data.drop(['Id', 'Artist Name', 'Track Name'], axis=1)
test_data = test_data.drop(['Artist Name', 'Track Name'], axis=1)

# Separate features and target
X = train_data.drop('Class', axis=1)
y = train_data['Class']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Handle missing values with SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

# Perform RandomizedSearchCV to find the best hyperparameters
grid_search = RandomizedSearchCV(rf_model, param_dist, cv=5, n_iter=10, random_state=42)
grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_

# Train the model with the best parameters
best_rf_model = RandomForestClassifier(random_state=42, **best_params)
best_rf_model.fit(X_train_scaled, y_train)

# Evaluate the model on the validation set
val_accuracy = best_rf_model.score(X_val_scaled, y_val)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: 0.521875


In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [45]:
# Load the data
train_data = pd.read_csv('train (1).csv')
test_data = pd.read_csv('test (2).csv')

In [46]:
# Check for any missing values
print(train_data.isnull().sum())

Id                       0
Artist Name              0
Track Name               0
Popularity             333
danceability             0
energy                   0
key                   1609
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      3541
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                    0
dtype: int64


In [47]:
# Save the 'Id' column from the test data
test_ids = test_data['Id']

X = train_data.drop('Class', axis=1)
y = train_data['Class']

In [48]:
# Define numerical and categorical columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

In [49]:
# Preprocessing pipeline for numerical columns
numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


In [50]:
# Preprocessing pipeline for categorical columns
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # You can use other strategies for categorical data
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [51]:
# Combine the preprocessing steps using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

In [52]:
# Transform the data using the preprocessor
X_processed = preprocessor.fit_transform(X)

In [53]:
# Initialize the model
model = RandomForestClassifier(random_state=42)

In [34]:
# Reduce the size of the dataset for hyperparameter tuning (optional)
X_tiny, _, y_tiny, _ = train_test_split(X, y, test_size=0.1, random_state=42)

In [35]:
# Transform the data using the preprocessor
X_tiny_processed = preprocessor.fit_transform(X_tiny)

In [36]:
# Initialize the model
model = RandomForestClassifier(random_state=42)

In [54]:
# Hyperparameter tuning using RandomizedSearchCV with reduced n_iter and smaller hyperparameter ranges
param_distributions = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2]
}

random_search = RandomizedSearchCV(model, param_distributions, n_iter=5, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
random_search.fit(X_tiny_processed, y_tiny)

print("Best parameters found: ", random_search.best_params_)
best_model = random_search.best_estimator_


Best parameters found:  {'n_estimators': 100, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_depth': None}


In [55]:
# Cross-validate the model
cv_accuracy = cross_val_score(best_model, X_processed, y, cv=5, scoring='accuracy')
print(f'Cross-validated accuracy: {np.mean(cv_accuracy):.2f}')

Cross-validated accuracy: 0.38


In [57]:
# Train the final model
best_model.fit(X_processed, y)

RandomForestClassifier(min_samples_leaf=2, min_samples_split=3, random_state=42)

In [1]:
# Scale the test data using the same preprocessor
X_test_processed = preprocessor.transform(test_data.drop('Id', axis=1))  # Drop 'Id' for prediction

NameError: name 'preprocessor' is not defined

In [ ]:
# Make predictions on the test set
test_predictions = best_model.predict(X_test_processed)

In [ ]:
# Create a DataFrame with the predictions and 'Id' column
submission_df = pd.DataFrame({'Id': test_data['Id'], 'Class': test_predictions})

In [18]:
# Save the predictions to a CSV file
submission_df.to_csv('music_genre_predictions.csv', index=False)


ValueError: Cannot use median strategy with non-numeric data:
could not convert string to float: 'Marina Maximilian'